# Step Two: Factor Screening (Forseeablility Analysis)

The purpose of this part is to rolling over different sectors to obtain the factor return's Spearman ranked ICs (the correlation between factor ranks and the future return). And then, screen the stable and significant factors in different industry sectors. Alphalens here is a fantastic toolkit to visualize the correlations as well as other computations. 

From the heatmap, we can see the historical turnover on such factor returns. **Stability is preferable.** The more stable, the more reliable of this factor.

From the average IC along separate sectors, we can see the significance of the factors along such sector. **Larger absolute value is appreciable.** The greater the absolute number, the more significant of this factor.

The data source is Chinese A market. The factors we tested are:
    
    * 'total_turnover'
    * 'volume'
    * 'market_cap'
    * 'a_share_market_val_2'
    * 'cash_received_from_sales_of_goods'
    * 'pb_ratio'
    * 'net_profit'
    * 'ps_ratio'
    
Time line: 01/01/2012 - 02/29/2016

In [1]:
from rqdata_utils import *
import pandas
import numpy as np
import scipy as sp
import alphalens as al
import talib
%matplotlib inline

ImportError: cannot import name 'factorial'

In [ ]:
price_df,instrument_df,equity_df = get_price_instrument_equity("cn_stock_price_2012_2018.csv","cn_instrument_info_2012_2018.csv","cn_equity_daily_2012_2018.csv","sectorCode")

In [ ]:
price_df.head()

In [ ]:
instrument_df.head()

In [ ]:
equity_df.head()

In [ ]:
Idx = pd.IndexSlice
equity_df.loc[Idx[:,'000001.XSHE'],'close'].head()

In [ ]:
from talib import abstract

In [ ]:
groups_dict = talib.get_function_groups()

In [ ]:
groups_dict['Cycle Indicators']

In [ ]:
for idx, group in equity_df["close"].groupby(level=1):
    print(group.values)
    output = talib.HT_(group.values,timeperiod=2)
#     print(talib.abstract.Function("HT_DCPERIOD"))
    print(output)
    break

In [ ]:
ht_dcpriod = talib.abstract.Function("HT_DCPERIOD")
plt.plot(group.values)
plt.plot(talib.HT_DCPERIOD(group.values))
plt.plot(talib.HT_TRENDLINE(group.values))
plt.plot(talib.HT_TRENDMODE(group.values))
plt.legend(["close","period","trendline","trendmode"])

In [ ]:
plt.plot(talib.HT_TRENDMODE(group.values))

In [ ]:
factor_columns = [equity_df.columns[i] for i in (2,3,7,8,9,10,11,12)]
factor_columns

In [ ]:
factor_list = []
for col in factor_columns:
#     print("factor: {}".format(col))
    factor_list.append(equity_df[col])
sectors = equity_df['sectorCode']

In [ ]:
# ic_list = []
# for my_factor in factor_list:
#     factor_data = al.utils.get_clean_factor_and_forward_returns(factor=my_factor,
#                                                             prices=price_df,
#                                                             groupby=sectors,
#                                                             periods=(1,21,63),
#                                                             max_loss=1)
#     ic = al.performance.mean_information_coefficient(factor_data,group_adjust=False,by_group=True,by_time=None)
#     print("#######################################################")
#     print("factor: {}".format(my_factor.name))
#     print(ic)
#     ic_list.append(al.performance.mean_information_coefficient(factor_data,group_adjust=False,by_group=True,by_time=None))

In [ ]:
# ic_list[0]

In [ ]:
# result = pd.concat(ic_list, keys=factor_columns)

In [ ]:
# result.index = result.index.set_names(['factor','group'])
# result

In [ ]:
# result.to_csv("factor_ic_analysis.csv")
# result2 = result.copy()

In [ ]:
ic_list = []
monthly_ic_list = []
for my_factor in factor_list:
    factor_data = al.utils.get_clean_factor_and_forward_returns(factor=my_factor,
                                                            prices=price_df,
                                                            groupby=sectors,
                                                            periods=(1,21,63),
                                                            max_loss=1)
    mean_ic = al.performance.mean_information_coefficient(factor_data,group_adjust=False,by_group=True,by_time=None)
    mean_monthly_ic = al.performance.mean_information_coefficient(factor_data, by_time='M')
    print("#######################################################")
    print("factor: {}".format(my_factor.name))
    print(mean_ic)
    al.plotting.plot_monthly_ic_heatmap(mean_monthly_ic)
    plt.show()
    ic_list.append(mean_ic)
    monthly_ic_list.append(mean_monthly_ic)


result = pd.concat(ic_list, keys=factor_columns)
result.index = result.index.set_names(['factor','group'])
result.to_csv("factor_ic_analysis.csv")

In [ ]:
from rqdata_utils import *
import pandas
import numpy as np
import scipy as sp
import alphalens as al
%matplotlib inline

price_df,instrument_df,equity_df = get_price_instrument_equity("cn_stock_price_2012_2018.csv",
                                                               "cn_instrument_info_2012_2018.csv",
                                                               "cn_equity_daily_2012_2018.csv",
                                                               "sectorCode")

factor_columns = [equity_df.columns[i] for i in (2,3,7,8,9,10,11,12)]
# mean_ic_df, monthly_ic_list = ic_analysis(equity_df, price_df, factor_columns,
#                                           group_column='sectorCode', periods=(1,22,66),
#                                          group_adjust=True)

In [ ]:
mean_ic_df, monthly_ic_list = ic_analysis(equity_df, price_df, factor_columns,group_column='sectorCode', periods=(1,22,66),group_adjust=True)
mean_ic_df.to_csv("factor_ic_analysis.csv")